<a href="https://colab.research.google.com/github/hellloooooo/Machine_learning/blob/main/%D0%A2%D1%80%D0%BE%D1%85%D0%B8%D0%BC%D0%B5%D1%86%D1%8C_%D0%BF%D1%80_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Трохимець М.В. ФІТ 4-8

Варіант 30

#Завдання 1

In [2]:
!pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 4.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357277 sha256=6cf09d93c115e56f5983340e00d74e0b08c3b8f2fc5dd0fd9f222ac29b723b3b
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise


In [3]:
from surprise import Dataset, Reader
from surprise import SVD, KNNBasic
from surprise.model_selection import cross_validate, GridSearchCV
from surprise import accuracy
from surprise.model_selection import train_test_split
import pandas as pd
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# 1. Завантажте датасет для рецензій (ml-100k) за допомогою бібліотеки
#Surprise.

data = Dataset.load_builtin('ml-100k')

# 2. Виведіть перші 5 рядків завантаженого датасету.
print("Перші 5 рядків датасету:")
for row in data.raw_ratings[:5]:
    print(row)

Dataset ml-100k could not be found. Do you want to download it? [Y/n] Y
Trying to download dataset from https://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /root/.surprise_data/ml-100k
Перші 5 рядків датасету:
('196', '242', 3.0, '881250949')
('186', '302', 3.0, '891717742')
('22', '377', 1.0, '878887116')
('244', '51', 2.0, '880606923')
('166', '346', 1.0, '886397596')


In [5]:
# 3. Реалізуйте два алгоритми для рекомендаційної системи на основі цього
#датасету. Можна вибрати будь-які алгоритми з бібліотеки Surprise.
# 4. Використайте крос-валідацію для підбору оптимальних параметрів для
# обох алгоритмів.


trainset, testset = train_test_split(data, test_size=0.2)

param_grid_svd = {
    'n_factors': [50, 100],
    'lr_all': [0.005, 0.010],
    'reg_all': [0.02, 0.05]
}

param_grid_knn = {
    'k': [20, 40],
    'min_k': [1, 5],
    'sim_options': {
        'name': ['cosine', 'pearson'],
        'user_based': [False, True]
    }
}

grid_search_svd = GridSearchCV(SVD, param_grid_svd, measures=['rmse', 'mae'], cv=5)
grid_search_svd.fit(data)

grid_search_knn = GridSearchCV(KNNBasic, param_grid_knn, measures=['rmse', 'mae'], cv=5)
grid_search_knn.fit(data)


Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing

In [6]:
# 5. Оберіть найкращий алгоритм на основі середньої абсолютної помилки
# (MAE).

best_svd = grid_search_svd.best_estimator['mae']
best_knn = grid_search_knn.best_estimator['mae']

print(f"Найкращий MAE для SVD: {grid_search_svd.best_score['mae']}")
print(f"Найкращий MAE для KNNBasic: {grid_search_knn.best_score['mae']}")

if grid_search_svd.best_score['mae'] < grid_search_knn.best_score['mae']:
    best_model = best_svd
    print("Вибраний алгоритм: SVD")
else:
    best_model = best_knn
    print("Вибраний алгоритм: KNNBasic")



Найкращий MAE для SVD: 0.7255438594481087
Найкращий MAE для KNNBasic: 0.8031918908057925
Вибраний алгоритм: SVD


In [7]:
# 6. Виведи рекомендації (10 фільмів) для конкретного користувача.

best_model.fit(trainset)

user_id = str(196)
user_ratings = trainset.ur[trainset.to_inner_uid(user_id)]
print(f"Кількість рецензій користувача {user_id}: {len(user_ratings)}")

all_items = set(trainset.all_items())
rated_items = set([item for (item, _) in user_ratings])
unrated_items = all_items - rated_items

predictions = [(item, best_model.predict(user_id, trainset.to_raw_iid(item)).est) for item in unrated_items]
predictions.sort(key=lambda x: x[1], reverse=True)

print("Топ-10 фільмів, рекомендованих для користувача:")
for item_id, rating in predictions[:10]:
    print(f"Фільм {trainset.to_raw_iid(item_id)} з прогнозованим рейтингом {rating:.2f}")

Кількість рецензій користувача 196: 31
Топ-10 фільмів, рекомендованих для користувача:
Фільм 408 з прогнозованим рейтингом 4.63
Фільм 1449 з прогнозованим рейтингом 4.61
Фільм 318 з прогнозованим рейтингом 4.61
Фільм 64 з прогнозованим рейтингом 4.59
Фільм 963 з прогнозованим рейтингом 4.54
Фільм 427 з прогнозованим рейтингом 4.52
Фільм 114 з прогнозованим рейтингом 4.48
Фільм 197 з прогнозованим рейтингом 4.47
Фільм 272 з прогнозованим рейтингом 4.45
Фільм 483 з прогнозованим рейтингом 4.43


**Висновок:** У ході цієї лабораторної роботи я реалізував два алгоритми для рекомендаційної системи на основі датасету "ml-100k", застосувавши крос-валідацію для обох алгоритмів, а також відобразив рекомендації  10 фільмів для певного користувача.